<a href="https://colab.research.google.com/github/yujin512/text_causal_inference/blob/master/predicting_pre_post_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving post_clean.csv to post_clean.csv


In [2]:
upload = files.upload()

Saving pre_clean.csv to pre_clean.csv


In [24]:
import pandas as pd
pre_df = pd.read_csv('pre_clean.csv')
post_df = pd.read_csv('post_clean.csv')

In [36]:
pre_df.merge(post_df, suffixes = ('_pre', '_post'), on=('id'))

,Unnamed: 0_pre,id,Figure 1 (Description)_pre,Figure 2 (Description)_pre,Figure 3 (Description)_pre,Unnamed: 0_post,Figure 1 (Description)_post,Figure 2 (Description)_post,Figure 3 (Description)_post
0,1,72,Figure one shows how a diamond's cut and its C...,This chart is much more elucidating. It shows ...,This set of charts is fairly confusing. All I ...,0,"Grandma, this graph is trying to show us that ...",There is a very clear (if non-linear!) relatio...,Diamonds come in all shapes and sizes.
1,2,97,"In Figure 1, we can see a substantial differen...","In Figure 2, we see the relationship between t...","In Figure 3, we see seven graphs depicting the...",2,The price of diamonds tends to vary based on c...,The price of diamonds tends to be positively a...,There is barely anything that can be explained...
2,4,32,"In the 0 to 1 carat range, fair cut diamonds h...","Overall, the price of a diamond increases as t...",This is a very confusing and poorly done graph.,1,Price is affected by Carat and Cut. Ideal cut ...,Carat is more significant in influencing price...,I still have no idea what's going on in this g...
3,5,15,This figure appears to describe the prices of ...,This figure uses different colors to different...,This figure allows us to see various trends ba...,3,I find this graph useful in its depiction of t...,This graph clearly shows that price increases ...,"This shows many disparate ideas, which don't r..."
4,9,83,Gems with higher carats sell for higher prices...,"Generally, the higher the carat the more a gem...",Gems with lower quality cuts tend to have a wi...,4,The greater the carat the more expensive the g...,Carat has a consistent effect on price -- the ...,There is a lot going on when it comes to the ...


For inspiration for features / metrics, I tried predicting the pre v. post label myself. 
To do this, I preprocessed the two datasets in the following manner:
1. Merge the two in a long form. 
2. Balance the dataset (there were twice as many pre responses as post -> add more post response. I should have cut the pre response to balance instead because the "repeated" obs gave away the label.)
3. Randomize the order of the observation.
4. Predict only on description (without figure, id, and the label). 

In [42]:
pre_df['workshop'] = ['pre']*len(pre_df)
post_df['workshop'] = ['post']*len(post_df)

In [84]:
cols = ['id', 'description', 'workshop']
long_df = pd.DataFrame(columns=cols)
for df in [pre_df, post_df]:
  for i in range(1, 4):
    this_res = df[['id', "Figure %s (Description)" % i, 'workshop']]
    this_res = this_res.rename(columns = {"Figure %s (Description)" % i: 'description'})
    long_df = pd.concat([long_df, this_res], join = 'outer')

# randomize the order   
long_df_rd = long_df.sample(len(long_df))

In [109]:
# balancing 
long_balanced = pd.concat([long_df_rd, long_df_rd[(long_df_rd['workshop'] == 'post')].sample(12)])
long_balanced_rd = long_balanced.sample(len(long_balanced))

In [111]:
for i in range(20):
  print(i, long_balanced_rd.iloc[i][1])

0 There is a lot going on when it comes to  the effects of carat, cut, and price. Cut does not have any effect on price but can have an effect on "value." Cuts are evenly distributed among differently weighted gems. More work is needed to understand the effects of carat and cut on a gem's price and value. 
1 It's hard to know what this figure is trying to say
2 The above chart shows how various variables or factors change based on the quality of cut. We can see how the cut affects the variables carat, depth, price, table, x, y, and z. Although the graph is a little bit difficult to look at.
3 This graph plots diamond carat vs. price, and it doesn't look like there's very much difference between the cut and the way that its price changes with size
4 There is a lot going on when it comes to  the effects of carat, cut, and price. Cut does not have any effect on price but can have an effect on "value." Cuts are evenly distributed among differently weighted gems. More work is needed to unde

In [117]:
my_guess_1 = ['pre', 'post', 'pre', 'pre', 'post', 'post', 'pre', 'post', 'pre', 'post', 
              'pre', 'post', 'post', 'pre', 'pre', 'pre', 'post', 'post', 'post', 'pre']
reason_1 = ['informative list', 'fatigue', 'overly general but made efforts (fig 3)', 'informative then insight', 'repeated obs', 
              'good abt ex', 'comparison, interesting point, why - not abt but enjoyable', 'terse', 'many insights but less focused', 
          'short, general explanation', 'description of plots not insights', 'repeated', 'abt + detailed', 'doesn\'t describe the relationship',
          'not abt but clear structure', 'describe the plots not insights', 'length', 'length', 'relationship','describe the plots - off topic']
scores_1 = [2, 2, 2, 2, 2, 4, 3, 3, 3, 2, 2, 4, 2, 4, 2, 2, 1, 2, 2, 2]

In [120]:
first_20 = pd.DataFrame(long_balanced_rd[:20])
first_20['guess'] = my_guess_1 
first_20['reasons'] = reason_1
first_20['score'] = scores_1

In [123]:
sum(first_20['workshop'] == first_20['guess'])/20

0.7

I surprisingly had a quite high accuracy (0.7) on 20 responses. I want to examine whether the accuracy is balanced across class. 


In [128]:
sum(first_20[first_20['workshop'] == first_20['guess']]['workshop'] == 'pre')

8

Out of 14 correct guesses, 8 were pre. So the prediction accuracy seems to be independent of the effect. 

In [129]:
first_20[first_20['workshop'] == first_20['guess']]

,id,description,workshop,guess,reasons,score
6,57,The above chart shows how various variables or...,pre,pre,overly general but made efforts (fig 3),2
7,60,"This graph plots diamond carat vs. price, and ...",pre,pre,informative then insight,2
4,83,There is a lot going on when it comes to the ...,post,post,repeated obs,2
5,63,Figure 1 shows boxplots for diamonds of differ...,pre,pre,"comparison, interesting point, why - not abt b...",3
4,83,The greater the carat the more expensive the g...,post,post,terse,3
0,72,This chart is much more elucidating. It shows ...,pre,pre,many insights but less focused,3
1,32,Price is affected by Carat and Cut. Ideal cut ...,post,post,"short, general explanation",2
7,60,This graph plots the distributions of prices f...,pre,pre,description of plots not insights,2
1,32,Price is affected by Carat and Cut. Ideal cut ...,post,post,repeated,4
2,8,The carat of a diamond -- whether between 0 an...,pre,pre,not abt but clear structure,2


A couple observations:
1. repeated observations are post workshop(procedural mistake, balance by cutting intead of sampling again next time)
2. post are shorter (contaminated - knew this going in) - fatigue, address in experimental design?
3. pre describes the plots themselves rather than insights in the plots - students perhaps understand what is being asked by questions after the workshop
4. responses with clear structure (even if it is not abt) are better regarded. 
